In [1]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 7.9 MB/s eta 0:00:00


In [2]:
import random
import numpy as np
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/cryoras/ND/refs/heads/main/ND.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Teks    5000 non-null   object
 1   Label   5000 non-null   object
dtypes: object(2)
memory usage: 78.3+ KB


In [8]:
data['Label'].unique()

array(['Gempa Bumi', 'Tsunami', 'Banjir', 'Tanah Longsor', 'UNIDENTIFIED'],
      dtype=object)

In [11]:
data_clean = data[data['Label'] != 'UNIDENTIFIED']

In [13]:
data_clean['Label'].unique()

array(['Gempa Bumi', 'Tsunami', 'Banjir', 'Tanah Longsor'], dtype=object)

# preprocessing text
1. removal spasial character
2. remove numeber
3. lower character
4. lemmatization

In [14]:
def preprocessed(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
  text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
  text = re.sub(r'RT[\s]', '', text) # menghapus RT
  text = re.sub(r"http\S+", '', text) # menghapus link
  text = re.sub(r'[0-9]+', '', text) # menghapus angka
  text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
  text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
  text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
  text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
  return text

def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

In [15]:
data_clean['Teks_phase'] = data_clean['Teks'].apply(preprocessed)
data_clean['Teks_clean'] = data_clean['Teks_phase'].apply(stemmingText)

<ipython-input-15-fd6d41bf8f3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Teks_phase'] = data_clean['Teks'].apply(preprocessed)
<ipython-input-15-fd6d41bf8f3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Teks_clean'] = data_clean['Teks_phase'].apply(stemmingText)


In [16]:
data_clean.head()

,Teks,Label,Teks_phase,Teks_clean
0,Warga Sumatera Barat panik akibat getaran kuat...,Gempa Bumi,Warga Sumatera Barat panik akibat getaran kuat...,warga sumatera barat panik akibat getar kuat g...
1,Terjadi gempa tektonik di Lampung dengan magni...,Gempa Bumi,Terjadi gempa tektonik di Lampung dengan magni...,jadi gempa tektonik di lampung dengan magnitud...
2,Terjadi gempa tektonik di Sulawesi Barat denga...,Gempa Bumi,Terjadi gempa tektonik di Sulawesi Barat denga...,jadi gempa tektonik di sulawesi barat dengan m...
3,Gempa bumi berkekuatan 8.4 melanda Nusa Tengga...,Gempa Bumi,Gempa bumi berkekuatan melanda Nusa Tenggara ...,gempa bumi kuat landa nusa tenggara barat deng...
4,Warga Sulawesi Tenggara panik akibat guncangan...,Gempa Bumi,Warga Sulawesi Tenggara panik akibat guncangan...,warga sulawesi tenggara panik akibat guncang h...
